In [7]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

load_dotenv(override=True) 

True

In [8]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search Agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")
)


In [9]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several advanced AI agent frameworks have emerged, enhancing the development and deployment of intelligent, autonomous systems. Notable among these are:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents and supporting complex interactions like multi-agent scenarios and dynamic workflows. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: This framework leverages LLMs and mathematical principles to design adaptive AI agents capable of real-time parameter reconfiguration. It defines a multidimensional parameter space to capture agent behaviors, supporting scalable, dynamic, and ethical AI applications. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system that introduces a standardized Agent-to-Agent communication layer based on the Model Context Protocol (MCP). It incorporates an Experience Pack architecture for structured knowledge retention and continual learning, enhancing adaptability and coordination in complex tasks. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development of advanced AI agents. It focuses on curating high-quality training data and novel strategies for agent test-time reflection and voting, achieving state-of-the-art results among open-source agents. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

Additionally, major tech companies have introduced their own AI agent frameworks:

- **OpenAI Agents SDK**: A lightweight Python framework released in March 2025, focusing on creating multi-agent workflows with comprehensive tracing and guardrails. It is compatible with over 100 different LLMs and offers detailed monitoring and debugging capabilities. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Google Agent Development Kit (ADK)**: Announced in April 2025, this modular framework integrates with Google's ecosystem, including Gemini and Vertex AI. It supports hierarchical agent compositions and requires minimal code for efficient development. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Amazon Bedrock AgentCore**: Unveiled at the AWS Summit New York 2025, AgentCore is a platform designed to simplify the development and deployment of advanced AI agents. It includes modular services supporting the full production lifecycle, emphasizing flexibility and scalability. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

These frameworks represent significant advancements in AI agent development, offering diverse tools and capabilities to meet the evolving demands of intelligent system deployment. 

In [ ]:
######## Structured output
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel): 
    reasoning: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel): 
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

plannerAgent = Agent(
    name = "PlannerAgent",
    instructions=INSTRUCTIONS, 
    model = "gpt-4o-mini",
    output_type=WebSearchPlan
)    

In [ ]:
####  Structured output. 
## Planner agent. 
## Email agent 
## Write agent 

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(plannerAgent, message)
    print(result.final_output)

searches=[WebSearchItem(reasoning='To find up-to-date frameworks in AI Agent development, I will look for recent publications or articles that specifically address frameworks released or popularized in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reasoning='Gathering insights from industry experts and news sources will provide a broader view of the AI landscape and highlight key frameworks being used in 2025.', query='2025 AI technology trends and frameworks'), WebSearchItem(reasoning='Exploring GitHub repositories will help identify open-source AI agent frameworks that have gained popularity or community support in 2025.', query='popular AI agent frameworks GitHub 2025')]


In [17]:

@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("jayantms@gmail.com")  # Change to your verified sender
    to_email = To("jayantms@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

    

In [18]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email Agent", 
    instructions=INSTRUCTIONS, 
    model="gpt-4o-mini", 
    tools=[send_email]
)


In [19]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

## define output types - 

class ReportData(BaseModel): 
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report") 
    follow_up_questions: list[str] = Field(description="Suggested topics to research further") 

writer_agent = Agent(
        name = "WriterAgent",
        instructions=INSTRUCTIONS, 
        model="gpt-4o-mini", 
        output_type=ReportData
    )

In [20]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches.")
    result = await Runner.run(plannerAgent, f"Query : {query}")
    print(f"Will perform {len(result.final_output.searches)} searches.")

    return result.final_output

async def perform_search(search_plan: WebSearchPlan): 
    """ Call search() for each item in the search plan """ 
    print("Searching..")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output


async def write_report(query: str, search_results: list[str]): 
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report") 
    return result.final_output 

async def email_report(reportdata: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email")
    result = await Runner.run(email_agent, reportdata.markdown_report)
    print("Email sent") 
    return result


In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"): 
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_search(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Search completed.")